In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as spst
import scipy.optimize as op
from cycler import cycler
from matplotlib.ticker import AutoMinorLocator
from matplotlib.ticker import MaxNLocator


<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)


In [2]:
# Define functions

def read_plate_reader(listfiles,interval=2):
    datalist=[]
    columnlist=[]
    for i in listfiles:
        df=pd.read_excel(i,sheet_name=0)
        datalist.append(df.replace('OVRFLW',100000).iloc[:,1:])
        columnlist.append(df.iloc[:,1:].columns.values)
    data=np.column_stack([datalist[i] for i in range(0,len(datalist))])
    columns=[item for sublist in columnlist for item in sublist]
    df1=pd.DataFrame(data,columns=columns)
    time=np.arange(0,df1.shape[0]*2,interval)/60
    return(time,df1)

def averagetimecourses(df,c):
    tmp=[]
    for i in c:
        tmp.append(df[i].values)
    outmean=np.mean(tmp,axis=0)
    outsd=np.std(tmp,axis=0)
    return (outmean,outsd)

def calibratePR():
    df=pd.read_csv('data/180121_EGFPcalibration.csv',delimiter=',') 
    
    xdat=df.iloc[:,0]
    ydat=df.iloc[:,3]
    w=df.iloc[:,4]

    def f(x,b):
        return b*x

    p0=0
    popt,pcov=op.curve_fit(f,xdat,ydat,p0,sigma=w,absolute_sigma=True)
    print(popt)
    perr = np.sqrt(np.diag(pcov))
    print(perr)
    b=popt[0]

def formatplot(ax,xlabel,ylabel,xlim=False,ylim=False,logx=False,logy=False,logxy=False,symlogx=False,padx=12,pady=12):

    ######### SET TITLES AND LABLES #########

    #ax.set_title('Plot title')
    if xlabel!=False:
        ax.set_xlabel(xlabel, labelpad=padx)
    if ylabel!=False:    
        ax.set_ylabel(ylabel, labelpad=pady)

    ######### SET AXES LIMITS #########

    if xlim!=False:
        ax.set_xlim(xlim)
    if ylim!=False:
        ax.set_ylim(ylim)

    ######### SET TICK VALUES #########

    #ax.set_xticks([0,0.5,1])
    #	ax.set_yticks([0,2,4,6,8])

    ######### SET LINE THICKNESSES #########

    #ax.xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter("%1.e"))
    #ax.axhline(linewidth=2, color='k')      
    #ax.axvline(linewidth=2, color='k')
    ax.spines['bottom'].set_linewidth(2)
    ax.spines['top'].set_linewidth(2)
    ax.spines['left'].set_linewidth(2)
    ax.spines['right'].set_linewidth(2) 

    ######### SET TICKS #########

    if logx==True:

        ax.set_xscale("log")

    elif logy==True:

        ax.set_yscale("log")

    elif logxy==True:

        ax.set_xscale("log")
        ax.set_yscale("log")
    
    elif symlogx==True:

        ax.set_xscale("symlog",linthreshx=1e-4)
        ax.set_yscale("log")

    else:
        minorLocatorx=AutoMinorLocator(2) # Number of minor intervals per major interval
        minorLocatory=AutoMinorLocator(2)
        ax.xaxis.set_minor_locator(minorLocatorx)
        ax.yaxis.set_minor_locator(minorLocatory)

    ax.tick_params(which='major', width=2, length=8, pad=9,direction='in',top='on',right='on')
    ax.tick_params(which='minor', width=2, length=4, pad=9,direction='in',top='on',right='on')

# Import plate reader data

In [3]:
# 1. Read data

listfilesMG100=['180904_MGGFP2_MG630_J.xlsx','180904_MGGFP2_MG630_K.xlsx','180904_MGGFP2_MG630_L.xlsx','180904_MGGFP2_MG630_M.xlsx','180904_MGGFP2_MG630_N.xlsx']
listfilesGFP50=['180904_MGGFP2_GFP50_J.xlsx','180904_MGGFP2_GFP50_K.xlsx','180904_MGGFP2_GFP50_L.xlsx','180904_MGGFP2_GFP50_M.xlsx','180904_MGGFP2_GFP50_N.xlsx']
listfilesGFP70=['180904_MGGFP2_GFP70_J.xlsx','180904_MGGFP2_GFP70_K.xlsx','180904_MGGFP2_GFP70_L.xlsx','180904_MGGFP2_GFP70_M.xlsx','180904_MGGFP2_GFP70_N.xlsx']
interval=2 # minutes

# 2. Append path
j=0
for i in listfilesMG100:
    listfilesMG100[j]='data/'+i
    j+=1

j=0
for i in listfilesGFP50:
    listfilesGFP50[j]='data/'+i
    j+=1
    
j=0
for i in listfilesGFP70:
    listfilesGFP70[j]='data/'+i
    j+=1
        
time,dfMG100=read_plate_reader(listfilesMG100,interval)
time,dfGFP50=read_plate_reader(listfilesGFP50,interval)
time,dfGFP70=read_plate_reader(listfilesGFP70,interval)
c0=np.array([21.33,10.665,5.3325,2.66625,1.333125,0.6665625,0.33328125,0.166640625])
c1=np.array([621.1,310.55,155.275,77.6375,38.81875,19.409375,9.7046875,4.85234375])

calibratePR()

# 3. Calibrate MG

c=[
    ['L9','M9','N13'],
    ['L10','M10'],
    ['L11','M11'],
    ['L12','M12'],
    ['L13','M13'],
    ['L14','M14'],
    ['L15','M15'],
    ['L16','M16']
]
c1=np.array([621.1,310.55,155.275,77.6375,38.81875,19.409375,9.7046875,4.85234375])
           

means=np.zeros((361,len(c)))
stds=np.zeros((361,len(c)))

for i in range(len(c)):
    outmean,outstd=averagetimecourses(dfMG100,c[i])
    means[:,i]=outmean
    stds[:,i]=outstd
bkg=np.mean(means[-10:-1,7]) # Estimate background


plt.close("all")

my_dpi=150

figure_options={'figsize':(8.3,5.83)} #figure size in inches. A4=11.7x8.3. A5=8.27,5.83
font_options={'size':'28','family':'sans-serif','sans-serif':'Arial'}
plt.rc('figure', **figure_options)
plt.rc('font', **font_options)

# Calibrate 
slope, intercept, r_value, p_value, std_err = spst.linregress(c1,means[0]-bkg)
print(slope,intercept,bkg)


# 4. Arrange data

c=[
    ['J9','K9','N9'],
    ['J10','K10'],
    ['J11','K11','N10'],
    ['J12','K12'],
    ['J13','K13','N11'],
    ['J14','K14'],
    ['J15','K15','N12'],
    ['J16','K16']
]
means=np.zeros((361,len(c)))
stds=np.zeros((361,len(c)))
for i in range(len(c)):
    outmean,outstd=averagetimecourses(dfGFP50,c[i])
    means[:,i]=outmean/1616.15775792
    stds[:,i]=outstd/1616.15775792
    
    
c=[
    ['J9','K9','N9'],
    ['J10','K10'],
    ['J11','K11','N10'],
    ['J12','K12'],
    ['J13','K13','N11'],
    ['J14','K14'],
    ['J15','K15','N12'],
    ['J16','K16']
]
means2=np.zeros((361,len(c)))
stds2=np.zeros((361,len(c)))
for i in range(len(c)):
    outmean2,outstd2=averagetimecourses(dfMG100,c[i])
    means2[:,i]=outmean2
    stds2[:,i]=outstd2
# Background subtract and convert. Requires you to run calibration code above first.
means2=(means2-bkg)/slope*1e-3
stds2=stds2/intercept*1e-3 

c=[
    ['L9','M9','N13'],
    ['L10','M10'],
    ['L11','M11'],
    ['L12','M12'],
    ['L13','M13'],
    ['L14','M14'],
    ['L15','M15'],
    ['L16','M16']
]
means3=np.zeros((361,len(c)))
stds3=np.zeros((361,len(c)))
for i in range(len(c)):
    outmean3,outstd3=averagetimecourses(dfGFP50,c[i])
    means3[:,i]=outmean3/1616.15775792
    stds3[:,i]=outstd3/1616.15775792

c=[
    ['L9','M9','N13'],
    ['L10','M10'],
    ['L11','M11'],
    ['L12','M12'],
    ['L13','M13'],
    ['L14','M14'],
    ['L15','M15'],
    ['L16','M16']
]
means4=np.zeros((361,len(c)))
stds4=np.zeros((361,len(c)))
for i in range(len(c)):
    outmean4,outstd4=averagetimecourses(dfMG100,c[i])
    means4[:,i]=outmean4
    stds4[:,i]=outstd4
# Background subtract and convert to uM
means4=(means4-bkg)/slope*1e-3
stds4=stds4/slope*1e-3


[1616.15775795]
[43.0781454]
8.536678298319273 11.575694982356481 76.27777777777777


# Plotly examples

In [4]:
from plotly.offline import iplot, init_notebook_mode
import plotly.io as pio
import plotly.graph_objs as go

import os

init_notebook_mode(connected=True)

Generate figure, which is a dictionary-like object. Then call offline plot to show it in jupyter.

In [5]:
trace1=go.Scatter(x=[1,2,3],y=[4,5,6],
                 marker={'color':'red', 'symbol':104, 'size':10},
                 mode='markers+lines',
                 text=['one','two','three'],
                 name='1st trace')

layout=go.Layout(title='First plot', xaxis={'title':'x1'},yaxis={'title':'x2'})

data=[trace1
     ]
figure=go.Figure(data=data,layout=layout)

iplot(figure,filename='pyguide_1')

In [6]:
figure

Update figure

In [7]:
figure.update(dict(layout=dict(title='Plot update')))
figure.update(data=[dict(marker=dict(color='blue'))])


iplot(figure,filename='pyguide_2')

In [8]:
# Box plots
y0=np.random.randn(50)
y1=np.random.randn(50)+1
box_1=go.Box(y=y0,name='Box1')
box_2=go.Scatter(y=y1,name='Box2',mode='markers')

data=[box_1,box_2]

layout=go.Layout(title='Box plots', xaxis={'title':'x1'},yaxis={'title':'x2'})

figbox=go.Figure(data=data,layout=layout)

iplot(figbox,filename='pyguide_3')

In [9]:
# Heat maps
z=[[1,20,30],[20,1,60],[30,60,1]]
heatmap=go.Heatmap(z=z,showscale=False)
data=[heatmap]

layout=go.Layout(title='Heatmap', xaxis={'title':'x1'},yaxis={'title':'x2'})

figheat=go.Figure(data=data,layout=layout)

iplot(figheat,filename='pyguide_4')

# Plotly working template

In [10]:
from plotly.offline import iplot, init_notebook_mode
import plotly.io as pio
import plotly.graph_objs as go

import os

init_notebook_mode(connected=True)

In [11]:
# Figure export

if not os.path.exists('images'):
    os.mkdir('images')

x=(np.arange(means2.shape[0])*2/60)
xrev=x[::-1]

current_palette=sns.color_palette("coolwarm", 9)
rgb=(np.array(current_palette)*255).astype(int)
rgbtuple=tuple([tuple(row) for row in rgb])

traces=[]
for i in range(means2.shape[1]):
    # Plot fill
    yupper=(means2[:,i]+stds2[:,i])
    ylower=(means2[:,i]-stds2[:,i])
    ylower=ylower[::-1]
    traces.append(go.Scatter(x=np.concatenate([x,xrev]),y=np.concatenate([yupper,ylower]),
                 fill='tozerox',
                 fillcolor='rgba'+str(rgbtuple[i])[:-1]+', 0.5)', # convert to rgba tuple
                 mode='none',
                 showlegend=False,
                 hoverinfo='none'
                 ))
    
    # Plot traces
    traces.append(go.Scatter(x=x,y=means2[:,i],
                 opacity=1,
                 marker={'symbol':0, 'size':8,'line':{'width':1},'color':'rgb'+str(rgbtuple[i])},
                 line={'width':2,'color':'rgb'+str(rgbtuple[i])},
                 mode='lines',
#                 text='Mean',
                 name="{:.1f}".format(c0[i])+' nM'))

    
layout=go.Layout(title='mRNA', 
                 titlefont={'family':'Arial','size':22},
                xaxis={'title':'Time (h)',
                       'titlefont':{'size':18},
                       'type':'linear',
                       'autorange':True,'range':[2,5], # Autorange overrides manual range
                       'ticks':'inside','zeroline':False,'ticklen':6,'tickwidth':2,
                       'tickfont':{'family':'Arial','size':18},
                      'showgrid':False,'showline':True,'mirror':'ticks','linewidth':2},
                yaxis={'title':'mRNA (uM)',
                       'titlefont':{'size':18},
                       'type':'linear',
                       'autorange':True,'range':[2,5], # Autorange overrides manual range
                       'ticks':'inside','zeroline':False,'ticklen':6,'tickwidth':2,
                       'tickfont':{'family':'Arial','size':18},
                      'showgrid':False,'showline':True,'mirror':'ticks','linewidth':2},
                autosize=False, width=891,height=630,
                margin=go.layout.Margin(l=100,r=100,b=100,t=100,pad=4), 
                showlegend=True,paper_bgcolor='rgba(0,0,0,0)',plot_bgcolor='rgba(0,0,0,0)')

figure=go.Figure(data=traces,layout=layout)
iplot(figure,filename='mRNA')
#pio.write_image(figure,'images/fig1.pdf')